## Update a DELWAQ model: forcing

Once you have a **DELWAQ** model, you may want to update your model in order to add new emission data, add sample locations, use different hydrological forcing data, create and run different scenarios etc.

With HydroMT, you can easily read your model and update one or several components of your model using the **update** function of the command line interface (CLI). Here are the steps and some examples on how to **update the model forcing**.

All lines in this notebook which starts with ! are executed from the command line. Within the notebook environment the logging messages are shown after completion. You can also copy these lines and paste them in your shell to get more feedback.

<font color=red>*This notebook supposes that you already run the **delwaq build EM** example notebook and that you have a **EM_test_full** D-Emissions model available in the examples folder.*</font>

### Import packages

In this notebook, we will use some functions of HydroMT to visualize available data. Here are the libaries to import.

In [1]:
import xarray as xr

In [2]:
# import hydromt
import hydromt

In [3]:
# setup logging
from  hydromt.log import setuplog
logger = setuplog("update_model_forcing", log_level=10)

2021-05-06 12:13:52,988 - update_model_forcing - log - INFO - HydroMT version: 0.4.0


### Hydrological forcing data from Wflow

D-Emissions or D-Water Quality models can use meteo/hydrological input data from [Wflow](https://deltares.github.io/Wflow.jl/stable/). The steps to get these data available for your model are:

* List the required meteo/hydro fluxes in the [Wflow TOML configuration file](https://deltares.github.io/Wflow.jl/stable/structure/#Config-and-TOML) in order to save them.
* Run the Wflow model.
* Add the outputs of the Wflow model as a new data source for HydroMT.
* Run hydroMT (specifically the [setup_hydrology_forcing component](https://deltares.github.io/hydromt_delwaq/latest/generated/hydromt_delwaq.delwaq.DelwaqModel.setup_hydrology_forcing.html)) to convert the wflow output to DELWAQ dynamic input.

More information on the meteo/hydrological forcing link between Wflow and DELWAQ is available in [docs(Wflow_outputs)](https://deltares.github.io/hydromt_delwaq/latest/advanced/coupling_wflow.html#hydrological-forcing-from-wflow).

#### Selection of the Wfow outputs

For our D-Emissions model, we will use hydrological data from the **wflow_piave** hydrologic model. For a D-Emissions model, the required meteo/hydrological inputs are:

* **precipitation**
* the amount of the precipitation that **infiltrates into the soil** from unpaved areas
* the amount of the precipation that goes directly to **surface runoff from paved areas**
* the amount of the precipation that goes directly to **surface runoff from unpaved areas**

In order to get these data from Wflow, you need to save these fluxes after the model run. These output fluxes are specified in the [Wflow TOML configuration file](https://deltares.github.io/Wflow.jl/stable/structure/#Config-and-TOML).

To set these options, you can either edit the TOML file manually or use HydroMT to help you do the trick using the `hydromt update` CLI API!

Here is the HydroMT **configuration file** needed to update the **Wflow** model:

In [4]:
fn_ini = 'wflow_update.ini'
with open(fn_ini, 'r') as f:
    txt = f.read()
print(txt)

[setup_config]                  # options parsed to wflow ini file <section>.<option>
starttime = 2010-02-02T00:00:00
endtime = 2010-02-10T00:00:00
timestepsecs = 86400
# Variables to save for a coupling between wflow and Delwaq (EM+WQ)
output.vertical.precipitation = "prec"
output.vertical.actinfilt = "inf"
output.vertical.excesswatersoil = "runUnp"
output.vertical.excesswaterpath = "runPav"
output.lateral.river.q_av = "q_river"
output.lateral.river.volume = "vol_river"
output.lateral.river.inwater = "inw_river"
output.lateral.land.q_av = "q_land"
output.lateral.land.volume = "vol_land"
output.lateral.land.inwater = "inw_land"
output.lateral.land.to_river = "inwint"


And the `hydromt update` CLI to use:

In [5]:
! hydromt update wflow wflow_piave -i wflow_update.ini -vvv

2021-05-06 12:13:55,125 - update - log - DEBUG - Writing log messages to new file /home/runner/work/hydromt_delwaq/hydromt_delwaq/docs/examples/examples/wflow_piave/hydromt.log.
2021-05-06 12:13:55,125 - update - log - INFO - HydroMT version: 0.4.0
2021-05-06 12:13:55,125 - update - main - INFO - Updating wflow model at /home/runner/work/hydromt_delwaq/hydromt_delwaq/docs/examples/examples/wflow_piave (r+).
2021-05-06 12:13:55,125 - update - main - INFO - Output dir: /home/runner/work/hydromt_delwaq/hydromt_delwaq/docs/examples/examples/wflow_piave
2021-05-06 12:13:55,125 - update - main - INFO - User settings:
2021-05-06 12:13:55,127 - update - cli_utils - INFO - setup_config.starttime: 2010-02-02T00:00:00
2021-05-06 12:13:55,127 - update - cli_utils - INFO - setup_config.endtime: 2010-02-10T00:00:00
2021-05-06 12:13:55,127 - update - cli_utils - INFO - setup_config.timestepsecs: 86400
2021-05-06 12:13:55,127 - update - cli_utils - INFO - setup_config.output.vertical.precipitation: pr

2021-05-06 12:13:55,220 - update - model_api - DEBUG - Model config read from /home/runner/work/hydromt_delwaq/hydromt_delwaq/docs/examples/examples/wflow_piave/wflow_sbm.toml
2021-05-06 12:13:55,220 - update - wflow - INFO - Read staticmaps from /home/runner/work/hydromt_delwaq/hydromt_delwaq/docs/examples/examples/wflow_piave/staticmaps.nc


2021-05-06 12:13:55,303 - update - model_api - DEBUG - Setting model config options.
2021-05-06 12:13:55,304 - update - wflow - INFO - Write model data to /home/runner/work/hydromt_delwaq/hydromt_delwaq/docs/examples/examples/wflow_piave
2021-05-06 12:13:55,304 - update - model_api - INFO - Writing model config to /home/runner/work/hydromt_delwaq/hydromt_delwaq/docs/examples/examples/wflow_piave/wflow_sbm.toml
2021-05-06 12:13:55,305 - update - wflow - INFO - Write staticmaps to /home/runner/work/hydromt_delwaq/hydromt_delwaq/docs/examples/examples/wflow_piave/staticmaps.nc


2021-05-06 12:13:55,869 - update - wflow - INFO - Writing model staticgeom to file.


The example above means the following: run **hydromt** with:

- `update wflow`: i.e. update a wflow model
- `wflow_piave_subbasin`: model folder to update
- `-i wflow_update_forcing.ini`: setup configuration file containing the components to update and their different options
- `-v`: give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.

The next step would then be to run the wflow_piave model using our updated TOML configuration file.

#### Available hydrological data from Wflow

For our D-Emissions model, we will use hydrological data from the **wflow_piave** hydrologic model.

The  model was already run beforehand and the **wflow_piave outputs** are stored in the *wflow_piave/run_default/output.nc* in NetCDF format. Let's have a look at the model outputs and see what data is available and for which periods.

Note: You can also inspect the file using [Panoply](https://www.giss.nasa.gov/tools/panoply/) or QGIS.

In [6]:
# Open the file
wflow_output_fn = 'wflow_piave/run_default/output.nc'
outputs = xr.open_dataset(wflow_output_fn, chunks={"time": 10})
#Print available variables and start and end time
print(f"Available outputs from wflow: {outputs.data_vars}")
times = outputs.time.values
print(f"Outputs available from {times[0]} to {times[-1]}")

Available outputs from wflow: Data variables: (12/13)
    runPav     (time, lat, lon) float32 dask.array<chunksize=(9, 53, 58), meta=np.ndarray>
    vol_river  (time, lat, lon) float32 dask.array<chunksize=(9, 53, 58), meta=np.ndarray>
    h_river    (time, lat, lon) float32 dask.array<chunksize=(9, 53, 58), meta=np.ndarray>
    inwint     (time, lat, lon) float32 dask.array<chunksize=(9, 53, 58), meta=np.ndarray>
    runUnp     (time, lat, lon) float32 dask.array<chunksize=(9, 53, 58), meta=np.ndarray>
    q_land     (time, lat, lon) float32 dask.array<chunksize=(9, 53, 58), meta=np.ndarray>
    ...         ...
    h_land     (time, lat, lon) float32 dask.array<chunksize=(9, 53, 58), meta=np.ndarray>
    inf        (time, lat, lon) float32 dask.array<chunksize=(9, 53, 58), meta=np.ndarray>
    vol_land   (time, lat, lon) float32 dask.array<chunksize=(9, 53, 58), meta=np.ndarray>
    q_river    (time, lat, lon) float32 dask.array<chunksize=(9, 53, 58), meta=np.ndarray>
    inw_land   (

We recognize from above all the variables and names we set previously in the **wflow_update.ini** file. All the required varibales are present and available for 9 days in February 2010.

#### Registering the wflow outputs as a hydroMT data source

As we are using hydrological forcing from a user defined run of the wflow model, the wflow output format and attributes can be different depending on the user settings. **Wflow outputs are then considered as local or user data source** and therefore need to be added to the HydroMT list of data sources using a **local yaml library file**.

Below you can see the **local_sources.yml** file corresponding to our wflow outputs:

In [7]:
fn_yml = 'local_sources.yml'
with open(fn_yml, 'r') as f:
    txt = f.read()
print(txt)

wflow_output:
  path: ./wflow_piave/run_default/output.nc
  crs: 4326
  data_type: RasterDataset
  driver: netcdf
  kwargs:
    chunks:
      lat: -1
      lon: -1
      time: 1
  meta:
    category: hydro
  rename:
    time: time
    prec: precip
    inf: infilt
    runPav: runPav
    runUnp: runUnp
    inw_river: inwaterRiv
    inw_land: inwaterLand
    inwint: inwaterInternal
    q_river: runRiv
    q_land: runLand
    vol_river: volRiv
    vol_land: volLand
  units:
    precip: mm
    infilt: mm
    runPav: mm
    runUnp: mm
    inwaterRiv: m3/s
    inwaterLand: m3/s
    inwaterInternal: m3/s
    runRiv: m3/s
    runLand: m3/s
    volRiv: m3
    volLand: m3


Here are some explanations about the file. You can have a look at the [HydroMT yaml data libary documentation](https://deltares.github.io/hydromt/latest/user_guide/data.html) for more information.

The first thing to define in the yaml library is the **name of the data source** you want to add in HydroMT. For example, here we use the name **wflow_output**. This name is important and is used in the HydroMT .ini file to tell HydroMT which data source you wish to use in a specific component. Once the name of the data source is set, the data attributes are listed:

* **path**: path to where the data is stored.
* **crs**: coordinate system of the data.
* **data_type**: `HydroMT DataCatalog type` either *RasterDataset* (gridded data), *GeoDataFrame* (vector data) or *GeoDataset* (point timeseries).
* **driver**: driver used to open the data. Either *raster* (GDAL compliant raster file), *netcdf* (NetCDF file), *zarr* (zarr file) or *vector* (GDAL compliant vector file).
* **kwargs**: optional arguments to read the data. Depends on the driver.
* **meta**: optional additional information on the data.
* **rename**: list used to rename the varibales inside of the data to HydroMT compliant names. The format is “name_in_dataset: name_in_HydroMT”. Note that the names present in the NetCDF file are the ones set up in the wflow TOML file. The list of standard HydroMT names for Delwaq forcing variables are: time, precip, infilt, runPav, runUnp, inwater (or inwaterLand, inwaterRiv), inwaterInternal, run (or runLand, runRiv), lev (or levLand, levRiv), vol (or volLand, volRiv).
* **units**: units of the variables in the data. Used by Delwaq only in order to convert from mm to m3/s and from m to m3 (requires complex unit conversion that just a multiplication or addition with a constant value).

### Model setup configuration

As with building, you can prepare a hydroMT **configuration file** that includes every components and settings that you want to run during your update.

The ini-file contains the model setup configuration and determines which components are updated and in which sequence and sets optional arguments for each component. This configuration is passed to hydromt using `-i <path_to_ini_file>`.

Each header as shown between `[...]` (e.g. `[setup_hydrology_forcing]`) corresponds with a model component which are explained in the [docs(model_components)](https://deltares.github.io/hydromt_wflow/latest/user_guide/wflow/components.html).

Let's open the example configuration file **delwaq_update_EM_forcing.ini** from the model repository [examples folder] and have a look at the settings.

In [8]:
fn_ini = 'delwaq_update_EM_forcing.ini'
with open(fn_ini, 'r') as f:
    txt = f.read()
print(txt)

[global]
mtype             = EM                    # type of Delwaq model ['EM', 'WQ']

[setup_hydrology_forcing]
hydro_forcing_fn  = wflow_output          # source name of the hydrological forcing in the yaml file
starttime         = 2010-02-03 00:00:00   # start time of the Delwaq run
endtime           = 2010-02-10 00:00:00   # end time of the Delwaq run
timestepsecs      = 86400                 # model timestep in seconds
add_volume_offset = True                  # add a one-timestep offset to the volume data in the hydrological forcing file compared to the flows



Here we can see that we will run **setup_hydrology_forcing** component to prepare daily hydrological forcing for 8 days in February 2010 using our **wflow_output**.

Note that the **add_volume_offset** function is set to *True*. This is because Delwaq needs water volumes at the beginning of the timestep. In some models, like wflow, volumes are written at the end of the timestep and therefore an offset of one timestep needs to be added for consistency.

You can find more information on the different components and their options in the [docs(model_components)](https://deltares.github.io/hydromt_delwaq/latest/user_guide/components.html).


### hydroMT CLI update interface

Using the `hydroMT update` API, we can update one or several components of an already existing Delwaq model. Let's get an overview of the available options:

In [9]:
# Print the options available from the update command
! hydromt update --help

Usage: hydromt update [OPTIONS] MODEL MODEL_ROOT

  Update a specific component of a model. Set an output directory to copy
  the edited model to a new folder, otherwise maps are overwritten.

  Example usage:
  --------------

  Update (overwrite) landuse-landcover maps in a wflow model
  hydromt update wflow /path/to/model_root -c setup_lulcmaps --opt source_name=vito

  Update reservoir maps based on default settings in a wflow model and write to new directory
  hydromt update wflow /path/to/model_root -o /path/to/model_out -c setup_reservoirs

Options:
  -o, --model-out DIRECTORY  Output model folder. Maps in MODEL_ROOT are
                             overwritten if left empty.

  -c, --components TEXT      Model components from ini file to run
  --opt TEXT                 Component specific keyword arguments, see the
                             setup_<component> method of the specific model
                             for more information about the arguments.

  -i, --config PA

### Update wflow forcing layers

In [10]:
# NOTE: copy this line (without !) to your shell for more direct feedback
! hydromt update delwaq ./EM_test_full -i delwaq_update_EM_forcing.ini -d local_sources.yml -vv

2021-05-06 12:14:00,984 - update - log - DEBUG - Appending log messages to file /home/runner/work/hydromt_delwaq/hydromt_delwaq/docs/examples/examples/EM_test_full/hydromt.log.
2021-05-06 12:14:00,985 - update - log - INFO - HydroMT version: 0.4.0
2021-05-06 12:14:00,985 - update - main - INFO - Updating delwaq model at /home/runner/work/hydromt_delwaq/hydromt_delwaq/docs/examples/examples/EM_test_full (r+).
2021-05-06 12:14:00,985 - update - main - INFO - Output dir: /home/runner/work/hydromt_delwaq/hydromt_delwaq/docs/examples/examples/EM_test_full
2021-05-06 12:14:00,985 - update - main - INFO - Additional data sources: ('/home/runner/work/hydromt_delwaq/hydromt_delwaq/docs/examples/examples/local_sources.yml',)
2021-05-06 12:14:00,985 - update - main - INFO - User settings:


2021-05-06 12:14:00,986 - update - cli_utils - INFO - global.mtype: EM
2021-05-06 12:14:00,986 - update - cli_utils - INFO - setup_hydrology_forcing.hydro_forcing_fn: wflow_output
2021-05-06 12:14:00,986 - update - cli_utils - INFO - setup_hydrology_forcing.starttime: 2010-02-03 00:00:00
2021-05-06 12:14:00,986 - update - cli_utils - INFO - setup_hydrology_forcing.endtime: 2010-02-10 00:00:00
2021-05-06 12:14:00,986 - update - cli_utils - INFO - setup_hydrology_forcing.timestepsecs: 86400
2021-05-06 12:14:00,986 - update - cli_utils - INFO - setup_hydrology_forcing.add_volume_offset: True
2021-05-06 12:14:00,991 - update - delwaq - INFO - Reading model staticgeom files.


2021-05-06 12:14:01,039 - update - delwaq - INFO - Read staticmaps from /home/runner/work/hydromt_delwaq/hydromt_delwaq/docs/examples/examples/EM_test_full/staticdata/staticmaps.nc
2021-05-06 12:14:01,061 - update - model_api - DEBUG - setup_hydrology_forcing.hydro_forcing_fn: wflow_output
2021-05-06 12:14:01,061 - update - model_api - DEBUG - setup_hydrology_forcing.starttime: 2010-02-03 00:00:00
2021-05-06 12:14:01,061 - update - model_api - DEBUG - setup_hydrology_forcing.endtime: 2010-02-10 00:00:00
2021-05-06 12:14:01,061 - update - model_api - DEBUG - setup_hydrology_forcing.timestepsecs: 86400
2021-05-06 12:14:01,061 - update - model_api - DEBUG - setup_hydrology_forcing.add_volume_offset: True
2021-05-06 12:14:01,061 - update - delwaq - INFO - Setting dynamic data from hydrology source wflow_output.
2021-05-06 12:14:01,072 - update - data_adapter - INFO - DataCatalog: Getting wflow_output RasterDataset netcdf data from /home/runner/work/hydromt_delwaq/hydromt_delwaq/docs/exampl

2021-05-06 12:14:01,108 - update - data_adapter - DEBUG - RasterDataset: Clip with geom - [11.775, 45.808, 12.742, 46.692]


2021-05-06 12:14:01,244 - update - delwaq - INFO - Write model data to /home/runner/work/hydromt_delwaq/hydromt_delwaq/docs/examples/examples/EM_test_full
2021-05-06 12:14:01,259 - update - delwaq - INFO - Writing staticmap files.
2021-05-06 12:14:01,273 - update - delwaq - INFO - Writing model staticgeom to file.


2021-05-06 12:14:01,291 - update - delwaq - INFO - Writing model config to file.
2021-05-06 12:14:01,292 - update - delwaq - WARNING - No updated maps. Skipping writing to file.
2021-05-06 12:14:01,320 - update - delwaq - INFO - Writing dynamicmap files.
2021-05-06 12:14:01,320 - update - delwaq - INFO - Writting dynamic data for timestep 1/8
2021-05-06 12:14:01,340 - update - delwaq - INFO - Writting dynamic data for timestep 2/8


2021-05-06 12:14:01,361 - update - delwaq - INFO - Writting dynamic data for timestep 3/8
2021-05-06 12:14:01,380 - update - delwaq - INFO - Writting dynamic data for timestep 4/8


2021-05-06 12:14:01,402 - update - delwaq - INFO - Writting dynamic data for timestep 5/8
2021-05-06 12:14:01,422 - update - delwaq - INFO - Writting dynamic data for timestep 6/8
2021-05-06 12:14:01,442 - update - delwaq - INFO - Writting dynamic data for timestep 7/8


2021-05-06 12:14:01,463 - update - delwaq - INFO - Writting dynamic data for timestep 8/8
2021-05-06 12:14:01,483 - update - delwaq - INFO - Writting waqgeom.nc file


The example above means the following: run **hydromt** with:

- `update delwaq`: i.e. update a delwaq model
- `./EM_test_full`: model folder
- `-i delwaq_update_EM_forcing.ini`: setup configuration file containing the components to update and their different options
- `-d local_sources.yml`: local data library, here containing the hydrological outputs from wflow.
- `-v`: give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.

### Visualization of the outputs

From the information above, you can see that the different forcing variables where updated. If you have a look at the output files, you can see that the hydrological data were added to the dynamicdata folder but also several information were added in the config folder.

In [11]:
import os
root = 'EM_test_full'
for path, _, files in os.walk(root):
    print(path)
    for name in files:
        if name.endswith('.xml'):
            continue
        print(f' - {name}')

EM_test_full
 - hydromt.log
EM_test_full/fews
EM_test_full/dynamicdata
 - hydrology.bin
EM_test_full/staticgeoms
 - monareas.geojson
 - basins.geojson
EM_test_full/staticdata
 - GHS-POP_2015.dat
 - staticmaps.nc
 - GHS-SMOD_2015.dat
 - slope.dat
 - streamorder.dat
 - GDP_world.dat
EM_test_full/hydromodel
 - rivwth.tif
 - ldd.tif
 - ptid.tif
 - basins.tif
 - modelmap.tif
 - rivlen.tif
EM_test_full/config
 - B4_nrofexch.inc
 - B2_nrofmon.inc
 - B2_stations.inc
 - B5_boundlist.inc
 - B1_timestamp.inc
 - B7_geometry-parameters.inc
 - B2_monareas.inc
 - B2_timers.inc
 - B7_geometry.inc
 - B2_outputtimes.inc
 - B3_attributes.inc
 - B7_geometry.bin
 - B7_hydrology.inc
 - B3_nrofseg.inc
 - B3_waqgeom.nc
 - B2_sysclock.inc
 - B2_timers_only.inc


You can have a look at some of these files:

In [12]:
import os
model_path = './EM_test_full'
fn_config = 'config/B1_timestamp.inc'
with open(os.path.join(model_path,fn_config), 'r') as f:
    txt = f.read()
print(txt)

'T0: 2010.02.03 00:00:00  (scu=       1s)'



For the hydrological data directly, the main files are:

* dynamicdata/hydrology.bin (Binary data)
* config/B7_hydrology.inc (Headers for the variables inside of hydrology.bin)

You can also see that a NetCDF file was created: **B3_waqgeom.nc**. This file can be used to produce NetCDF outputs directly when running D-Emissions or D-Water Quality but also to visualize the model in FEWS.

In [13]:
import os
model_path = './EM_test_full'
fn_config = 'config/B7_hydrology.inc'
with open(os.path.join(model_path,fn_config), 'r') as f:
    txt = f.read()
print(txt)

SEG_FUNCTIONS
Rainfall RunoffPav RunoffUnp Infiltr TotalFlow 

